# Sequence Manipulation & Iteration

**Role:** Senior Python Engineer

**Context:** Advanced Sequence Handling & Functional Programming Patterns

## Overview

Efficient data manipulation requires mastery of Python's sequence protocol. While loops and indices are primitive tools, Python provides high-level built-in functions—`sorted`, `reversed`, `slice`, and `iter`—to handle sequences (lists, tuples, strings) expressively and efficiently.

This module explores how to manipulate order, decouple slicing logic, and manually control iteration flows.

---

## 1. Non-Destructive Sorting: `sorted()`

Unlike the `list.sort()` method, which sorts a list **in-place** (modifying the original data), `sorted()` returns a **new list**, leaving the original sequence untouched. It works on *any* iterable, not just lists.

### Syntax

```python
sorted(iterable, key=None, reverse=False)

```

### Advanced Usage: The `key` Argument

The `key` parameter expects a callable (function) that transforms each element before comparison. This is critical for sorting complex objects like dictionaries or class instances.

**Algorithm Note:** Python uses **Timsort**, a hybrid sorting algorithm derived from merge sort and insertion sort. It is **stable**, meaning that if two elements have equal keys, their original order is preserved.

### Engineering Example: Sorting Complex Data

Sorting a list of server logs based on priority levels defined in a custom order.

```python
logs = [
    {'id': 101, 'msg': 'Disk full', 'level': 'CRITICAL'},
    {'id': 102, 'msg': 'User login', 'level': 'INFO'},
    {'id': 103, 'msg': 'Timeout',    'level': 'WARNING'},
    {'id': 104, 'msg': 'DB slow',    'level': 'CRITICAL'}
]

# Define priority weight (lower is more urgent)
priority_map = {'CRITICAL': 0, 'WARNING': 1, 'INFO': 2}

# Sort by priority using a lambda that looks up the weight
sorted_logs = sorted(logs, key=lambda x: priority_map.get(x['level'], 99))

print("Sorted by Priority:")
for log in sorted_logs:
    print(f"[{log['level']}] {log['msg']}")

```

---

## 2. Lazy Reversal: `reversed()`

The `reversed()` function returns a **reverse iterator**, not a list. This is a memory-efficient approach known as "lazy evaluation." It does not copy the data; it simply yields items from the tail to the head.

### Syntax

```python
reversed(sequence)

```

### Memory Efficiency: `reversed()` vs Slicing `[::-1]`

* **Slicing (`seq[::-1]`):** Creates a full copy of the list in memory. Fast, but memory-intensive for large datasets.
* **`reversed(seq)`:** Creates a lightweight iterator. Zero memory overhead for the data itself.

### Engineering Example: Processing Large Datasets

When processing a 1GB log file from the end (most recent logs) to the beginning.

```python
# Simulating a large dataset
time_series_data = range(1000000)

# 1. EFFICIENT: Using an iterator
# No new list is created in memory
rev_iter = reversed(time_series_data)

# We can consume the iterator one by one
last_item = next(rev_iter)
print(f"Last Item: {last_item}")

# 2. MATERIALIZING: Converting to list (Only do this if necessary)
# This consumes memory to store the reversed copy
small_list = [1, 2, 3, 4, 5]
rev_list = list(reversed(small_list))
print(f"Reversed List: {rev_list}")

```

---

## 3. Reusable Slicing: `slice()`

While the `[start:stop:step]` syntax is standard, Python exposes the underlying mechanism via the `slice()` built-in. This allows you to name and reuse slice logic, effectively treating "slicing strategies" as variables.

### Syntax

```python
slice(start, stop, step)

```

### Engineering Example: Fixed-Width File Parsing

In legacy systems (like Mainframes) or binary protocols, data often comes in fixed-width strings rather than JSON. Hardcoding integers makes code unreadable. Named slices solve this.

```python
raw_record = "ID9982  2023-10-27  ERROR_CODE_404"

# Define reusable slice strategies (Configuration)
# instead of raw_record[0:8], raw_record[8:20]...
IDX_ID = slice(0, 8)
IDX_DATE = slice(8, 20)
IDX_ERROR = slice(20, None) # From index 20 to end

# Apply slices
record_id = raw_record[IDX_ID].strip()
record_date = raw_record[IDX_DATE].strip()
error_msg = raw_record[IDX_ERROR].strip()

print(f"Parsed: ID={record_id}, Date={record_date}, Error={error_msg}")

```

---

## 4. Manual Iteration: `iter()` and `next()`

The `for` loop in Python is syntax sugar. Under the hood, it calls `iter()` to get an iterator and `next()` to retrieve values. You can interact with this protocol manually for fine-grained control.

### Syntax

```python
iter(iterable)
next(iterator, default)

```

### The `StopIteration` Exception

When an iterator is exhausted, it raises `StopIteration`. A `for` loop catches this internally to stop the loop. When using `next()` manually, you must handle this or provide a **default** value.

### Engineering Example: Custom Control Flow

Imagine a polling system where you want to fetch exactly 3 items from a stream, or wait for a specific condition.

```python
tasks = ["Task_A", "Task_B", "Task_C"]

# 1. Create the iterator
task_iterator = iter(tasks)

# 2. Manually consume items
print(f"Processing: {next(task_iterator)}") # Task_A
print(f"Processing: {next(task_iterator)}") # Task_B

# 3. Safe consumption with default (avoids crash on empty)
# If the iterator is empty, return "No Tasks" instead of raising StopIteration
next_task = next(task_iterator, "No Tasks")
print(f"Processing: {next_task}") # Task_C

final_check = next(task_iterator, "Queue Empty")
print(f"Status: {final_check}") # Queue Empty

```

### Advanced: Sentinel Iteration

`iter()` has a two-argument form: `iter(callable, sentinel)`. It calls the function repeatedly until it returns the sentinel value.

```python
import random

def get_status():
    return random.choice(["RUNNING", "RUNNING", "DONE", "ERROR"])

# Call get_status() until it returns "DONE"
print("--- Monitoring Job ---")
for status in iter(get_status, "DONE"):
    print(f"Current Status: {status}")
print("Job Finished.")

```